In [1]:
import pandas as pd
import numpy as np 
import scipy
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from xgboost import XGBRFClassifier
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.feature_selection import SequentialFeatureSelector
from mlxtend.feature_selection import sequential_feature_selector
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import Pipeline
%matplotlib inline
import pickle
from matplotlib import pyplot

In [2]:
full_data = pd.read_csv('daibetes.csv')
X = pd.read_csv('Scaled_X_new1.csv')
y = full_data['diabetes']

f_1 = pd.concat([X, y], axis=1).reset_index(drop=True)
f_1

#print(np.__version__)
#print(pd.__version__) 

#y.value_counts()
y.value_counts(normalize=True)


full_data.tail(10)

full_data['diabetes'].value_counts()
#full_data[full_data['diabetes'] == 1]
full_data



,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


In [7]:
#X.drop(columns='Unnamed: 0', inplace=True)

full_data

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


In [ ]:
rus = RandomUnderSampler(sampling_strategy='majority')
x_under, y_under = rus.fit_resample(X, y)
y_under.value_counts()

ros = RandomOverSampler(sampling_strategy='minority')
x_over, y_over = ros.fit_resample(X, y)
y_over.value_counts()

over = SMOTE(sampling_strategy=0.6)
under = RandomUnderSampler(sampling_strategy=0.9)
step = [('o', over), ('u', under)]
a = Pipeline(steps=step)
xim, yim = a.fit_resample(X, y)
yim.value_counts()
#y.value_counts()

y_over.value_counts()

In [ ]:

xgb_rf = XGBRFClassifier()

x_train, x_test, y_train, y_test = train_test_split(x_under, y_under, train_size=0.2, random_state=23)

xgb_rfm = xgb_rf.fit(x_train, y_train)

#XGpre = xgb_rfm.predict(x_test)
#confusion_matrix(XGpre, y_test)
#print(classification_report(XGpre, y_test))
#pickle.dump('xgb_rfm', open('Xgtry.pkl', 'wb'))

In [ ]:
ros = RandomOverSampler(sampling_strategy='minority')
x_over, y_over = ros.fit_resample(X, y)
y_over.value_counts()

In [ ]:

criterion = ['gini', 'entropy'] 
max_features = ['log2', 'sqrt']
min_samples_split = np.arange(2, 10) 
param = dict(criterion=criterion, max_features=max_features,min_samples_split=min_samples_split)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=4)


dt = DecisionTreeClassifier()
varmodel = GridSearchCV(estimator=dt, param_grid=param, cv=cv, error_score='raise')
feat_select = varmodel.fit(x_under, y_under)
feat_select.best_params_

dt = DecisionTreeClassifier(criterion='entropy', max_features='log2', min_samples_split= 9)
feat_select1 = dt.fit(X, y)
a = feat_select1.feature_importances_
feature_imp = pd.Series(a, index=X.columns).sort_values(ascending=False)

feature_imp
feature_imp.plot.bar()

In [ ]:
xgbrf = XGBRFClassifier()

xgbrfm = xgbrf.fit(X, y)
xgbrfm_Fea_im = xgbrfm.feature_importances_


xgb_rfm_fea_im1 = pd.Series(xgbrfm_Fea_im, index=X.columns).sort_values(ascending=False)

#a = pd.DataFrame(xgb_rfm_Fea_im1).reset_index()
#b = a.rename(columns={0:'Values', 'index': 'Names'}, inplace=True)


xgb_rfm_fea_im1
xgb_rfm_fea_im1.plot.pie()

xgb_rfm_fea_im1

In [5]:
X = X.loc[:, ['HbA1c_level', 'blood_glucose_level','age',
'bmi','hypertension']]


rus = RandomUnderSampler(sampling_strategy='majority')
X, y = rus.fit_resample(X, y)
y.value_counts(normalize=True)


diabetes
0    0.5
1    0.5
Name: proportion, dtype: float64

In [24]:
X = X.loc[:, ['HbA1c_level', 'blood_glucose_level','age',
'bmi','hypertension']]


ros = RandomOverSampler(sampling_strategy='minority')
x_over, y_over = ros.fit_resample(X, y)
y_over.value_counts()


diabetes
0    91500
1    91500
Name: count, dtype: int64

In [ ]:
#RFC

x_train, x_test, y_train, y_test =  train_test_split(X, y, test_size=0.3, random_state=88)


N_est = np.arange(1, 40)
cri = ('gini', 'entropy', 'log_loss')
max_fe = ('sqrt', 'log2')

cv = RepeatedStratifiedKFold(n_splits=8, n_repeats=3, random_state=99)

grid = dict(n_estimators=N_est, criterion=cri, 
            max_features=max_fe)
rf_model = RandomForestClassifier()

hyper_para_gr = GridSearchCV(estimator=rf_model, param_grid=grid, 
                             cv=cv, n_jobs=5, error_score='raise')
hyper_para_gr


h_model = hyper_para_gr.fit(x_train, y_train)
h_model.score(x_test, y_test)
predict_Val = h_model.predict(x_test)
#accuracy_score(predict_Val, y_test)
confusion_matrix(predict_Val, y_test)
print(classification_report(predict_Val, y_test))


In [ ]:
cal_model = CalibratedClassifierCV(hyper_para_gr, method='sigmoid', cv=cv, n_jobs=-1)
cal_model  = cal_model.fit(x_train, y_train)
cal_P = cal_model.predict(x_test)
print(classification_report(cal_P, y_test))
a , b = calibration_curve(y_test, cal_P, n_bins=6)

pyplot.title('Calibration plot')
pyplot.xlabel('Mean pred probs(positive class:1)')
pyplot.ylabel('fraction of positive(positive class:1)')
pyplot.plot([0, 1], [0, 1], linestyle='--')
pyplot.plot(a, b, linestyle='-.')


In [ ]:
#SVM Model 

x_train, x_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=88)

svm_m= svm.SVC()

H_param = {'C': [0.1, 10, 100],
'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
'gamma':['scale', 'auto']}
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=99)

RanGD_m = RandomizedSearchCV(estimator=svm_m, param_distributions=H_param, cv=cv, n_jobs=1, error_score='raise')



svm_m1 = RanGD_m.fit(x_train, y_train)


#svm_m1.score(X_test, y_test)
#svM_predict = svm_m1.predict(X_test)
#accuracy_score(svm_predict, y_test)
#confusion_matrix(svm_predict, y_test)
#print(classification_report(svm_predict, y_test))


In [ ]:
#Evaluate Model

svm_m1.score(x_test, y_test)
svm_predict = svm_m1.predict(x_test)
accuracy_score(svm_predict, y_test)
confusion_matrix(svm_predict, y_test)
print(classification_report(svm_predict, y_test))
cross_val_score(svm_m1, x_train, y_train, cv=cv)

In [ ]:
cal_svm = CalibratedClassifierCV(svm_m1, method='sigmoid', cv=cv)

c_svm = cal_svm.fit(x_test, y_test)
c_P_svm = c_svm.predict(x_test)

print(classification_report(c_P_svm, y_test))
a, b = calibration_curve(c_P_svm, y_test, n_bins=7)

pyplot.title('Calibration plot')
pyplot.xlabel('Mean pred probs(positive class:1)')
pyplot.ylabel('Fraction of positive(positive class:1)')
pyplot.plot([0, 1], [0, 1], linestyle='--')
pyplot.plot(a, b, linestyle=':')


In [ ]:
pickle.dump(Svm_M1, open('SVM.pkl', 'wb'))

In [26]:
m = pickle.load(open('Dai_RfM.pkl', 'rb'))
from sklearn.model_selection import train_test_split

X = X.loc[:, ['HbA1c_level', 'blood_glucose_level','age',
'bmi','hypertension']]

ros = RandomOverSampler(sampling_strategy='minority')
x_over, y_Over = ros.fit_resample(X, y)
y_Over.value_counts()

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=88)


predicted_value = m.predict(x_test)
accuracy_score(predicted_value, y_test)
confusion_matrix(predicted_value, y_test)
print(classification_report(predicted_value, y_test))

for n, i in enumerate(predicted_value):
    print(n, i)

X.iloc[72069:]

              precision    recall  f1-score   support

           0       0.67      1.00      0.80     18533
           1       0.97      0.22      0.35     11467

    accuracy                           0.70     30000
   macro avg       0.82      0.61      0.58     30000
weighted avg       0.79      0.70      0.63     30000

0 1
1 0
2 0
3 0
4 1
5 1
6 1
7 0
8 0
9 0
10 1
11 1
12 1
13 0
14 0
15 1
16 0
17 0
18 0
19 1
20 1
21 0
22 1
23 0
24 0
25 0
26 0
27 1
28 1
29 0
30 1
31 0
32 0
33 1
34 0
35 1
36 0
37 1
38 0
39 1
40 0
41 1
42 1
43 0
44 1
45 0
46 0
47 1
48 0
49 1
50 0
51 0
52 0
53 1
54 0
55 1
56 0
57 0
58 1
59 0
60 1
61 1
62 0
63 0
64 0
65 1
66 0
67 0
68 0
69 0
70 0
71 1
72 1
73 0
74 0
75 1
76 1
77 0
78 1
79 0
80 0
81 1
82 1
83 0
84 0
85 0
86 0
87 1
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 1
99 1
100 1
101 0
102 0
103 1
104 0
105 0
106 0
107 1
108 0
109 0
110 1
111 0
112 0
113 1
114 0
115 0
116 1
117 1
118 0
119 0
120 0
121 1
122 1
123 0
124 1
125 1
126 0
127 0
128 0
129 0
130

,HbA1c_level,blood_glucose_level,age,bmi,hypertension
72069,0.496228,0.432895,0.1750,0.465059,0.0
72070,0.591657,0.000000,0.4750,0.465059,0.0
72071,0.591657,0.332996,0.1750,0.465059,0.0
72072,0.438971,0.332996,0.1750,0.465059,0.0
72073,0.419886,0.443994,0.3750,0.662527,0.0
...,...,...,...,...,...
99995,0.515314,0.055499,1.0000,0.465059,0.0
99996,0.572571,0.110999,0.0250,0.197737,0.0
99997,0.419886,0.416245,0.8250,0.478760,0.0
99998,0.095429,0.110999,0.3000,0.682677,0.0


In [44]:
db_new = pd.read_csv('db_new.csv')


db_new.iloc[72069:]

X = X.loc[:, ['HbA1c_level', 'blood_glucose_level','age',
'bmi','hypertension']]


import pickle
m = pickle.load(open('Dai_RfM.pkl','rb'))


a = 0
b = 5.8
c = 140
d = 27.32
e = 14

aa = (a - db_new['hypertension'].min())/(db_new['hypertension'].max() - db_new['hypertension'].min())

bb = (b - db_new['HbA1c_level'].min())/(db_new['HbA1c_level'].max() - db_new['HbA1c_level'].min())

cc = (c - db_new['blood_glucose_level'].min())/(db_new['blood_glucose_level'].max() - db_new['blood_glucose_level'].min())

dd = (d - db_new[' bmi'].min())/(db_new[' bmi'].max() - db_new[' bmi'].min())

ee = (e - db_new['age'].min())/(db_new['age'].max() - db_new['age'].min())


print(aa, bb, cc, dd, ee)

#db_new.iloc[72069:]

m.predict([[bb, cc, ee, dd, aa]])


0.0 0.4389712344038383 0.33299577154945614 0.46505858051640486 0.175


c:\Users\Yusuf\Documents\Analytical and technical projects\Diabetes analytical solutions\daipred\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0], dtype=int64)

In [46]:
db_new.iloc[72069:]


,gender,age,hypertension,heart_disease,smoking_history,diabetes,HbA1c_level,bmi,blood_glucose_level
72069,Male,14.0,0,0,No Info,0,6.1,27.32,158.0
72070,Female,38.0,0,0,never,0,6.6,27.32,80.0
72071,Female,14.0,0,0,not current,0,6.6,27.32,140.0
72072,Female,14.0,0,0,never,0,5.8,27.32,140.0
72073,Female,30.0,0,0,not current,0,5.7,34.67,160.0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,0,6.2,27.32,90.0
99996,Female,2.0,0,0,No Info,0,6.5,17.37,100.0
99997,Male,66.0,0,0,former,0,5.7,27.83,155.0
99998,Female,24.0,0,0,never,0,4.0,35.42,100.0


In [ ]:
m2 = pickle.load(open('SVM.pkl', 'rb'))

predicted_value2 = m2.predict(x_test)

accuracy_score(predicted_value2, y_test)
confusion_matrix(predicted_value2, y_test)
print(classification_report(predicted_value2, y_test))